In [1]:
import pandas as pd
import numpy as np

In [3]:
#!pip install --upgrade pip
#!pip install --upgrade tensorflow
#!pip install --upgrade theano
#!pip install --upgrade keras
#!pip install --upgrade keras-text
#!pip install --upgrade tensorflow-gpu

In [4]:
import google.datalab.storage as storage
import pandas as pd
from io import BytesIO

mybucket = storage.Bucket('bharti_patel')
data_csv = mybucket.object('merged2_data.csv')

uri = data_csv.uri
%gcs read --object $uri --variable data
df = pd.read_csv(BytesIO(data), index_col=0, parse_dates=True, infer_datetime_format=True)
df.head()


mybucket = storage.Bucket('bharti_patel')
data2_csv = mybucket.object('merged_data.csv')

uri = data2_csv.uri
%gcs read --object $uri --variable data2
df2 = pd.read_csv(BytesIO(data2), index_col=0, parse_dates=True, infer_datetime_format=True)
df.head()

,Close,Volume,change_after_3_days,change_after_2_days,change_after_1_days,binary_sentiment_after_3_days,sentiment_after_3_days,binary_sentiment_after_2_days,sentiment_after_2_days,binary_sentiment_after_1_days,sentiment_after_1_days,title_clean
2018-05-31,24415.839844,429740000,5.54548,-0.175886,2.518083,pos,negative,neg,positive,pos,negative,racial bias in bail decisions
2018-05-31,24415.839844,429740000,5.54548,-0.175886,2.518083,pos,negative,neg,positive,pos,negative,to hit canada with steel aluminum tariffs at m...
2018-05-31,24415.839844,429740000,5.54548,-0.175886,2.518083,pos,negative,neg,positive,pos,negative,walmart perk for workers go to college for day
2018-05-31,24415.839844,429740000,5.54548,-0.175886,2.518083,pos,negative,neg,positive,pos,negative,why china looming debt problems will not stop ...
2018-05-31,24415.839844,429740000,5.54548,-0.175886,2.518083,pos,negative,neg,positive,pos,negative,italy financial crisis explained


In [5]:
df = df.dropna()

In [6]:
df = df.groupby([ pd.Grouper(freq='D'), 'sentiment_after_3_days', 'binary_sentiment_after_3_days'])['title_clean'].apply(list).reset_index()
df.head()

,level_0,sentiment_after_3_days,binary_sentiment_after_3_days,title_clean
0,2008-03-31,negative,pos,"[silver gold the last american hero jfk, resou..."
1,2008-04-01,positive,neg,"[copper pipes now worth more than some homes, ..."
2,2008-04-02,positive,neg,[bush caused the financial market meltdown]
3,2008-04-04,negative,pos,"[trucker view of the us economy, la boy whirlp..."
4,2008-04-11,negative,pos,[dollar falls below yuan for first time since ...


In [7]:
x = df.title_clean
y = df.sentiment_after_3_days
z = df.binary_sentiment_after_3_days
u = df2.title_clean

MAX_SENT_LENGTH = 1000
MAX_SENTS = 50
MAX_NB_WORDS = 100000




In [8]:
from keras import backend; print(backend._BACKEND)
from keras import backend as K
import importlib
import os
def set_keras_backend(backend):
    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        importlib.reload(K)
        assert K.backend() == backend
print ("Change Keras Backend to tensorflow")        
set_keras_backend("tensorflow")  
from keras import backend; print(backend._BACKEND)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


tensorflow
Change Keras Backend to tensorflow
tensorflow


In [9]:
# define baseline model
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Activation
from keras.layers.embeddings import Embedding
from keras.layers import TimeDistributed
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, BatchNormalization, GRU
from keras.layers import Flatten
from keras.layers import Reshape
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer , text_to_word_sequence
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model
from keras.engine.topology import Layer, InputSpec
#import keras_metrics
import tensorflow as tf
from keras import initializers
from keras import backend as K
#import theano as th
from keras.utils import multi_gpu_model

In [10]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

In [11]:
precision = as_keras_metric(tf.metrics.precision)
recall = as_keras_metric(tf.metrics.recall)

In [12]:


tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(u)
sequences = tokenizer.texts_to_sequences(u)

vocab_size = len(u)



/usr/local/envs/py3env/lib/python3.5/site-packages/keras_preprocessing/text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [13]:
word = 'microfinance'
tokenizer.word_index[word]
#x = x.str.replace('microfinace', 'microfinance')

16859

In [14]:

#from keras_text.processing import SentenceWordTokenizer

data = np.zeros((vocab_size, MAX_SENTS, MAX_SENT_LENGTH), dtype='float64')

print (data.shape)
  


for i, sentences in enumerate(x):
    for j, sent in enumerate(sentences):
        #if j< MAX_SENTS:
         wordTokens = text_to_word_sequence(sent)
         k=0
         for _, word in enumerate(wordTokens):
              #if k<MAX_SENT_LENGTH and tokenizer.word_index[word]<MAX_NB_WORDS:
               #print (word)
               #print(tokenizer.word_index[word])
               if word == 'microfinace':
                  word = 'microfinance'
                  data[i,j,k] = tokenizer.word_index[word]
               k=k+1                    
               
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))


(1310, 50, 1000)
Total 61210 unique tokens.


In [15]:
enc = LabelEncoder()
onehot = OneHotEncoder(sparse=False)
seq_label = enc.fit_transform(y)
#seq_label = seq_label.reshape(len(seq_label), 1)
#seq_label_hot = onehot.fit_transform(seq_label)
print('Shape of seq_label tensor:', seq_label.shape)
print(seq_label[0])

Shape of seq_label tensor: (1310,)
0


In [16]:
from numpy import asarray
from numpy import zeros
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


In [17]:
# find the embedding vector dimensions size from the most common word in english: 'the'
# http://www.dictionary.com/e/commonwords/
embedding_size = len(embeddings_index['the'])
print (embedding_size)

100


In [18]:
embedding_matrix = np.random.random((len(word_index) + 1, embedding_size))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            embedding_size,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True)



In [19]:
print('Shape of embedding matrix:', embedding_matrix.shape)

Shape of embedding matrix: (61211, 100)


In [20]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
train_pct = 0.8
test_pct = 1 - train_pct
validation_pct = 0.5
SEED = 2000
seq_label = to_categorical(seq_label)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
seq_label = seq_label[indices]
nb_test_validation_samples = int(test_pct * data.shape[0])

x_train = data[:-nb_test_validation_samples]
y_train = seq_label[:-nb_test_validation_samples]
x_val_test = data[-nb_test_validation_samples:]
y_val_test = seq_label[-nb_test_validation_samples:]

nb_test_samples = int(validation_pct * x_val_test.shape[0])

x_test = x_val_test[:-nb_test_samples]
y_test = y_val_test[:-nb_test_samples]
x_val = x_val_test[-nb_test_samples:]
y_val = y_val_test[-nb_test_samples:]


print("Training sequences shape:", x_train.shape)
print("Training labels shape:",y_train.shape)
print("Test sequences shape:", x_test.shape)
print("Test labels shape:", y_test.shape)
print("Validation sequences shape:", x_val.shape)
print("Validation labels shape:", y_val.shape)

Training sequences shape: (1049, 50, 1000)
Training labels shape: (1049, 3)
Test sequences shape: (131, 50, 1000)
Test labels shape: (131, 3)
Validation sequences shape: (130, 50, 1000)
Validation labels shape: (130, 3)


In [21]:
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
encoder = TimeDistributed(sentEncoder)(input)
l_lstm_sent = Bidirectional(LSTM(100))(encoder)
preds = Dense(3, activation='softmax')(l_lstm_sent)
model = Model(input, preds)
#parallel_model = multi_gpu_model(model, gpus=None)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print("model fitting - Hierachical LSTM")
model.summary()

model fitting - Hierachical LSTM
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50, 1000)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 200)           6281900   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               240800    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 603       
Total params: 6,523,303
Trainable params: 6,523,303
Non-trainable params: 0
_________________________________________________________________


In [22]:
# fit the model

model.fit(x_test, y_test, epochs=50, verbose=0,validation_data=(x_val, y_val))


In [23]:
# evaluate the model
#loss, accuracy, precision, recall = model.evaluate(x_test, y_test, verbose=0)
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)


In [24]:
#print('Accuracy: %f, precision: %f, recall: %f, f1_score: %f' % ((accuracy), (precision), (recall), (2 * precision * recall / (precision + recall))))
print('Accuracy: %f' % (accuracy))

Accuracy: 0.580153


In [28]:
input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='float64')
embedded_sequences = embedding_layer(input)

encoder = TimeDistributed(embedding_layer)(input)

bigram_branch = TimeDistributed(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))(encoder)
bigram_branch = TimeDistributed(GlobalMaxPooling1D())(bigram_branch)
trigram_branch = TimeDistributed(Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1))(encoder)
trigram_branch = TimeDistributed(GlobalMaxPooling1D())(trigram_branch)
fourgram_branch = TimeDistributed(Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1))(encoder)
fourgram_branch = TimeDistributed(GlobalMaxPooling1D())(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch])


LSTM_out = Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2))(merged)

output = (Dense(3, activation='sigmoid'))(LSTM_out)

model2 = Model(inputs=[input], outputs=[output])

model2.compile(optimizer='rmsprop', loss='categorical_crossentropy',
          metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 50, 1000)     0                                            
__________________________________________________________________________________________________
time_distributed_18 (TimeDistri (None, 50, 1000, 100 6121100     input_9[0][0]                    
__________________________________________________________________________________________________
time_distributed_19 (TimeDistri (None, 50, 999, 100) 20100       time_distributed_18[0][0]        
__________________________________________________________________________________________________
time_distributed_21 (TimeDistri (None, 50, 998, 100) 30100       time_distributed_18[0][0]        
__________________________________________________________________________________________________
time_distr

In [29]:
# fit the model

model2.fit(x_test, y_test, epochs=50, verbose=0,validation_data=(x_val, y_val))

In [30]:
# evaluate the model
loss, accuracy, precision, recall = model2.evaluate(x_test, y_test, verbose=0)


In [31]:
print('Accuracy: %f, precision: %f, recall: %f, f1_score: %f' % ((accuracy), (precision), (recall), (2 * precision * recall / (precision + recall))))

Accuracy: 0.496183, precision: 0.450382, recall: 0.900763, f1_score: 0.600509
